In [17]:
import pandas as pd
import numpy as np
import glob
import os

In [18]:
df_wiv = pd.read_csv('results/regressions/city_sec_iv.csv')
df_ckols = pd.read_csv('results/regressions/city_occ_ols.csv')
df_cols = pd.read_csv('results/regressions/deltas/city_wage_ols.csv')

In [19]:
rho = 1 - (1 / df_wiv['iv_wage_coeff'])
rho = rho.unique()[0]

In [20]:
lambda_k = df_ckols[['occ_year', 'occ_year_coeff']].groupby('occ_year').mean()
lambda_k = - lambda_k

t_ckt = df_ckols[['occ_year', 'city_year', 'residuals']]
t_ckt['residuals'] = t_ckt['residuals'] * (1 - rho)
t_ckt['t_ckt'] = t_ckt['residuals'].apply(lambda x: np.exp(x))
t_ckt = t_ckt.drop(columns=['residuals'])
t_ckt.to_csv('results/params/t_ckt.csv', index=False)

/tmp/ipykernel_288958/1212370624.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t_ckt['residuals'] = t_ckt['residuals'] * (1 - rho)
/tmp/ipykernel_288958/1212370624.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t_ckt['t_ckt'] = t_ckt['residuals'].apply(lambda x: np.exp(x))


In [21]:
files = glob.glob('../data/processed/city_occ_employment/*.csv')

employment = pd.DataFrame()

for file in files:
    year = int(os.path.basename(file)[-8:-4])
    current = pd.read_csv(file)
    current['city_total'] = current.iloc[:, 1:].sum(axis=1)
    current = current.melt(id_vars=['MET2013'], var_name='Occupation', value_name='Employed')
    current['Year'] = year
    employment = pd.concat([employment, current], ignore_index=True)

employment = employment.groupby(['Occupation', 'Year']).sum().reset_index()
employment = employment[employment['Occupation'] != 'city_total']
employment = employment.drop(columns=['MET2013'])

year_total = employment.groupby('Year').sum().reset_index()
employment = employment.merge(year_total, on='Year', suffixes=('', '_total'))
employment['Occupation_Share'] = employment['Employed'] / employment['Employed_total']
employment = employment.drop(columns=['Employed', 'Employed_total', 'Occupation_total'])

normalization = employment[employment['Occupation'] == 'occ2_management'].rename(columns={'Occupation_Share': 'Occupation_Share_normalization'})
employment = employment.merge(normalization[['Year', 'Occupation_Share_normalization']], on='Year')
employment['normalized'] = employment['Occupation_Share'] / employment['Occupation_Share_normalization']
employment['occ_year'] = '(' + employment['Occupation'] + ')' + '_' + employment['Year'].astype(str)
employment = employment[['occ_year', 'normalized']]

In [22]:
lambda_k = lambda_k.merge(employment, on='occ_year')

lambda_k['year'] = lambda_k['occ_year'].apply(lambda x: int(x[-4:]))
numerator = lambda_k.groupby('year').sum().reset_index()
numerator = numerator.rename(columns={'normalized': 'numerator'})

lambda_k = lambda_k.merge(numerator[['year', 'numerator']], on='year')
lambda_k['T_k'] = lambda_k['numerator'] / lambda_k['occ_year_coeff']
T_k = lambda_k[['occ_year', 'T_k']]
T_k.to_csv('results/params/T_k.csv', index=False)

In [28]:
T_c = df_cols[['city_year', 'city_year_coeff']].groupby('city_year').mean()
T_c['rho'] = rho
T_c['T_c'] = T_c['city_year_coeff'] * (1 - T_c['rho'])
T_c['T_c'] = T_c['T_c'].apply(lambda x: np.exp(x))
T_c = T_c.drop(columns=['city_year_coeff', 'rho'])
T_c.to_csv('results/params/T_c.csv', index=True)

In [24]:
theta = df_cols['wage_coeff'].unique()[0]
theta = - theta * (1 - rho)

major_params = pd.DataFrame({'rho': [rho], 'theta': [theta]})
major_params.to_csv('results/params/major_params.csv', index=False)